In [ ]:
%matplotlib qt

In [ ]:
import matplotlib
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np 
import sqlite3
from itertools import product
from io import StringIO
import pathlib

In [ ]:
plt.rcParams['text.usetex'] = True
fig_output_dir="Figures"
pathlib.Path(fig_output_dir).mkdir(parents=True, exist_ok=True) 

## Explicitly added data from Perlmutter logs

In [ ]:
data_samples_sel="""Nodes_num	GPU_num	Num_Qubits	Timing(second)
1	2	33	19.9864
1	4	34	27.5023
2	8	35	75.8404
4	16	36	121.0355
8	32	37	164.8277
16	64	38	206.6019
32	128	39	248.1001
64	256	40	300.3545
128	512	41	343.8438"""


data_adjoint_sel="""Nodes_num	GPU_num	Num_Qubits	Timing(second)	layers
64	256	37	2304.71028	1
32	128	36	1916.874307	1
16	64	35	1541.559073	1
8	32	34	1164.236925	1
4	16	33	845.7922967	1
2	8	32	754.2725837	1
1	4	31	675.4695403	1"""

In [ ]:
data_samples_sel_perlmutter = pd.read_csv(StringIO(data_samples_sel), delimiter="\t")
data_adjoint_sel_perlmutter = pd.read_csv(StringIO(data_adjoint_sel), delimiter="\t")

In [ ]:
data_samples_sel_perlmutter

In [ ]:
data_adjoint_sel_perlmutter

## Plot samples of SEL

In [ ]:
cmap = matplotlib.colormaps['tab20']
colors = [cmap(0)]
plt.rcParams.update({'font.size': 16})

plt.figure()
x = data_samples_sel_perlmutter["Num_Qubits"]
g = data_samples_sel_perlmutter["Nodes_num"]
y = data_samples_sel_perlmutter["Timing(second)"]
labels = [f"$\\textrm{{{x0}}}$ " + f"$({{{l0}}}/{{{l0*4}}})$" for x0,l0 in zip(x,g)]
plt.bar(x,y, color=colors[0], linewidth=2, ec='black')
plt.xticks(x, labels, rotation=45,  ha='right', rotation_mode='anchor')
plt.xlabel("$\\textrm{qubits (nodes/gpus)}$")
plt.ylabel("$t~\\textrm{(s)}$", fontdict={"size" : 16})
plt.yscale('log')
plt.ylim([1e1, 5e2])
plt.tight_layout()
plt.savefig(f"{fig_output_dir}/LGPU_SampleSEL_MPI_Perlmutter.pdf", dpi=200)
plt.close("all")

## Plot adjoint grad of SEL

In [ ]:
cmap = matplotlib.colormaps['tab20']
colors = [cmap(0)]
plt.rcParams.update({'font.size': 16})

plt.figure()
x = data_adjoint_sel_perlmutter["Num_Qubits"]
g = data_adjoint_sel_perlmutter["Nodes_num"]
y = data_adjoint_sel_perlmutter["Timing(second)"]
labels = [f"$\\textrm{{{x0}}}$ " + f"$({{{l0}}}/{{{l0*4}}})$" for x0,l0 in zip(x,g)]
plt.bar(x,y, color=colors[0], linewidth=2, ec='black')
plt.xticks(x, labels,  rotation=45,  ha='right', rotation_mode='anchor')
plt.xlabel("$\\textrm{qubits (nodes/gpus)}$")
plt.ylabel("$t~\\textrm{(s)}$", fontdict={"size" : 16})
plt.yscale('log')
plt.ylim([5e2, 3e3])
plt.tight_layout()
plt.savefig(f"{fig_output_dir}/LGPU_AdjointSEL_MPI_Perlmutter.pdf", dpi=200)
plt.close("all")

In [ ]:
cmap = matplotlib.colormaps['tab20']
colors = [cmap(0)]
plt.rcParams.update({'font.size': 16})

In [ ]:
f, axs = plt.subplots(2,1, sharex=False, squeeze=True)

for idx,(dat,ax) in enumerate(zip([data_samples_sel_perlmutter, data_adjoint_sel_perlmutter], axs)):
    x = dat["Num_Qubits"]
    g = dat["Nodes_num"]
    y = dat["Timing(second)"]
    labels = [f"$\\textrm{{{x0}}}$ " + f"$({{{l0}}}/{{{l0*4}}})$" for x0,l0 in zip(x,g)]
    ax.bar(x,y, color=cmap(1+idx*4), linewidth=2, ec='black', label="$10^4$ samples" if idx ==0 else "Gradient")
    ax.set_xticks(x, labels, rotation=45,  ha='right', rotation_mode='anchor')
    if idx == 1:
        ax.set_xlabel("$\\textrm{qubits (nodes/gpus)}$",  fontdict={"size" : 20})
    ax.set_ylabel("$t~\\textrm{(s)}$", fontdict={"size" : 20})
    ax.set_yscale('log')
    ax.set_ylim([1e1, 5e2] if idx == 0 else [5e2, 3e3])

In [ ]:
plt.tight_layout()

In [ ]:
plt.tight_layout()
plt.subplots_adjust(wspace=0.1, hspace=0.7)

In [ ]:
fig = plt.figure()
offsets = np.linspace(-0.2, 0.2, 2)
width = offsets[1] - offsets[0]
x_values = set()
g_values = set()
for idx,dat in enumerate([data_samples_sel_perlmutter,data_adjoint_sel_perlmutter]):
    x = dat["Num_Qubits"]
    g = dat["Nodes_num"]
    y = dat["Timing(second)"]
    x_values.update(x)
    g_values.update(g)
    plt.bar(x+offsets[idx], y, color=cmap(idx*2), width = width, linewidth=2, ec='black', label="$10^4$ samples" if idx ==0 else "Jacobian")
    if idx == 1:
        ax.set_xlabel("$\\textrm{qubits (nodes/gpus)}$")
    plt.ylabel("$t~\\textrm{(s)}$") #, fontdict={"size" : 16})
    plt.yscale('log')

x_values=list(x_values)
g_values=list(g_values)

print(x_values, g_values)

labels = [f"$\\textrm{{{x0}}}$" for x0 in x_values]
plt.xticks(x_values, labels, rotation=45,  ha='right', rotation_mode='anchor')

plt.legend()
plt.tight_layout()

In [ ]:
plt.savefig(f"{fig_output_dir}/LGPU_SamplesAndAdjointSEL_MPI_Perlmutter.pdf", dpi=200)